Code History:<br>
1. Version 1.0 (2023/03/09):<br>
    - Base version, working as expected

<strong>Features:</strong><br>
- Scrape corporate and government bonds summary and details<br>
<br>
Plan: Data is scraped <strong>every weekday on 6PM GMT+7</strong>, few hours after the market has closed for the day. So the data you see before 6PM is previous trading day data.

In [1]:
import json
from json.decoder import JSONDecodeError
import numpy as np
import pandas as pd
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import queue
import threading
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import dateparser

import os
import sqlalchemy
from sqlalchemy import create_engine

# Chrome Selenium Starter<br>
<br>
Why Selenium? Because I need it to bypass cloudfare restriction

Initialize the Chrome driver

In [2]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

# Scrape Bond Summary

## BEI Bonds List

In [3]:
urls = {
    'Corporate Bond':'https://www.idx.co.id/secondary/get/BondSukuk/bond?pageSize=10000&indexFrom=1&bondType=1',
    'Goverment Bond':'https://www.idx.co.id/secondary/get/BondSukuk/bond?pageSize=10000&indexFrom=1&bondType=2'  
}

In [4]:
BEIBondsListDF = pd.DataFrame()
for issuer_type in urls:
    print(issuer_type)
    driver.get(urls[issuer_type])
    WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.TAG_NAME, 'body'))
    BEIBondsListContent = driver.find_element(By.TAG_NAME, value='body').text
    BEIBondsTypeListDF = pd.DataFrame(json.loads(BEIBondsListContent)['Results']).drop(columns='Nomor')
    BEIBondsTypeListDF['IssuerType'] = issuer_type
    
    BEIBondsListDF = pd.concat([BEIBondsListDF, BEIBondsTypeListDF])
    
BEIBondsListDF['MatureDate'] = pd.to_datetime(BEIBondsListDF['MatureDate']).dt.normalize()

Corporate Bond
Goverment Bond


In [5]:
BEIBondsListDF

BondId                                           BondName IssuerCode  \
0    ABLS01XXMF              MTN Asian Bulk Logistics I Tahun 2022     ABLS-M   
1       ABSM01C  Obligasi AB Sinar Mas Multifinance I Tahun 202...       ABSM   
2       ADCP01B  Obligasi I Adhi Cummuter Properti Tahun 2021 S...       ADCP   
3       ADCP02A  Obligasi II Adhi Commuter Properti Tahun 2022 ...       ADCP   
4       ADCP02B  Obligasi II Adhi Commuter Properti Tahun 2022 ...       ADCP   
..          ...                                                ...        ...   
183      VR0091     Obligasi Negara Republik Indonesia Seri VR0091       GOVT   
184      VR0092     Obligasi Negara Republik Indonesia Seri VR0092       GOVT   
185      VR0093     Obligasi Negara Republik Indonesia Seri VR0093       GOVT   
186      VR0094     Obligasi Negara Republik Indonesia Seri VR0094       GOVT   
187      VR0095     Obligasi Negara Republik Indonesia Seri VR0095       GOVT   

    MatureDate Rating   Outstanding      IssuerType  
0   2027-06-21   None  1.000000e+12  Corporate Bond  
1   2023-09-04   irA-  3.500000e+10  Corporate Bond  
2   2024-05-20  idBBB  9.000000e+09  Corporate Bond  
3   2023-05-31  idBBB  2.055000e+11  Corporate Bond  
4   2025-05-24  idBBB  1.020000e+11  Corporate Bond  
..         ...    ...           ...             ...  
183 2030-12-29   None  2.334195e+12  Goverment Bond  
184 2027-12-29   None  2.981024e+13  Goverment Bond  
185 2028-12-29   None  2.981024e+13  Goverment Bond  
186 2029-12-29   None  2.981024e+13  Goverment Bond  
187 2030-12-29   None  2.981024e+13  Goverment Bond  

[1335 rows x 7 columns]

## Close and Quit Driver

In [6]:
driver.quit()

# Scrape Bond Details

## Get Bond Details Function

 Well, the website has a weird issue, i can access medium term notes with url intended for corporate / govt bonds<br>
 MTN example: https://www.ksei.co.id/services/registered-securities/medium-term-notes/lc/ABLS01XXMF<br>
 Different URL example: https://www.ksei.co.id/services/registered-securities/corporate-bonds/lc/ABLS01XXMF<br>
 Try it and you can still access the medium term notes<br>
'https://www.ksei.co.id/services/registered-securities/medium-term-notes/lc/ABLS01XXMF'<br>
'https://www.ksei.co.id/services/registered-securities/government-bonds/lc/FR0037'

In [7]:
def get_bond_details(BondId):
    while True:
        try:
            url = 'https://www.ksei.co.id/services/registered-securities/corporate-bonds/lc/' + BondId
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            data = {}

            # Find the dl tag with class="deflist deflist--with-colon"
            dl_tag = soup.find('dl', class_='deflist deflist--with-colon')

            # Loop through all dt tags within the dl tag and get their text values
            dt_tags = dl_tag.find_all('dt')
            for dt in dt_tags:
                # Get the text value of the dt tag
                dt_text = dt.get_text(strip=True)
                # Get the corresponding dd tag and its text value
                # find_next_sibling is actually an important function and it's new for me xD
                dd_text = dt.find_next_sibling('dd').get_text(strip=True)
                # Add the dd_text to the data dictionary with the dt_text as the key
                data[dt_text] = dd_text
            break
        except:
            time.sleep(1.5)
    
    time.sleep(2)
    return data

## Multithreading with Progress Bar

## Load Previous Scraped Data

In [8]:
engine = create_engine(
    "postgresql://{}:{}@{}/{}".format(
        os.getenv('POSTGRE_USER'), os.getenv('POSTGRE_PW'), os.getenv('POSTGRE_HOST'), os.getenv('POSTGRE_DB')
    )
)
conn = engine.connect()

NameError: name 'create_engine' is not defined

In [ ]:
try:
    prev_bond_details_df = pd.read_sql('SELECT * FROM BondDetails', con=conn)
except:
    prev_financial_report_df = pd.read_excel('bonds.xlsx')
    print('BEIBondsListDF DB Not Available')

## Create List to Store Scraped Data

In [ ]:
df_list = []

In [ ]:
with tqdm(total=len(BEIBondsListDF['BondId'])) as pbar:
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        
        for BondId in BEIBondsListDF['BondId']:
            if BondId in prev_bond_details_df['BondID']:
                continue
            else:
                future = executor.submit(get_bond_details, BondId)
                futures.append(future)

        # Use tqdm to add a progress bar to the multithreading process
        for future in as_completed(futures):
            pbar.update(1)
            df_list.append(future.result())

## Join All Bond Details and Cleaning

### Join Bond Details

In [ ]:
BondDetailsDF = pd.DataFrame(df_list)
BondDetailsDF

In [ ]:
BondDetailsDF.columns

### Data Transformation<br>
<br>
1. Some dates are written in Indonesian format string, 'May' is written as 'Mei', so I use dateparser library to tackle this issue and convert it to pandas datetime column<br>
2. Interest rate format is string, convert it to float32<br>
3. Replace '-' string with NaN

In [ ]:
BondDetailsDF['Listing Date'] = BondDetailsDF['Listing Date'].apply(lambda x: dateparser.parse(x) if x != '-' else np.nan)
BondDetailsDF['Mature Date'] = BondDetailsDF['Mature Date'].apply(lambda x: dateparser.parse(x) if ((x != '-') and (type(x) == str)) else np.nan)
BondDetailsDF['Effective Date ISIN'] = BondDetailsDF['Effective Date ISIN'].apply(lambda x: dateparser.parse(x) if x != '-' else np.nan)
BondDetailsDF['Interest/Disc Rate'] = BondDetailsDF['Interest/Disc Rate'].replace('%', '', regex=True).apply('float32')
BondDetailsDF = BondDetailsDF.replace('-', np.nan)

In [ ]:
BondDetailsDF.describe(include='all')

### Drop Unnecessary Columns<br>
<br>
1. Every column dropped has mostly missing value

In [ ]:
BondDetailsDF = BondDetailsDF.drop(columns=['Current Amount', 'Effective Date ISIN', 'Day Count Basis', 'Exercise Price'])

# Export Results

In [ ]:
BondDetailsDF['LastScraped'] = datetime.now()
BondDetailsDF = pd.concat([prev_bond_details_df, BondDetailsDF])
BondDetailsDF

## Export to Excel

In [ ]:
# BondDetailsDF.to_excel('bonds.xlsx', index=False)

## Export to DB

In [ ]:
BondDetailsDF.to_sql('BondDetails', con=conn, if_exists='replace', index=False)